In [1]:
# libraries
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import (confusion_matrix, accuracy_score, classification_report, roc_curve, auc)

import plotly.figure_factory as ff
from keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
from keras.models import Model, load_model
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.applications import VGG16
from tensorflow.keras.callbacks import LearningRateScheduler


import numpy as np

In [2]:
labels = ['alert', 'slight_drowsy','mod_drowsy','very_drowsy'] # alert, slightly drowsy, moderately drowsy and very drowsy

In [5]:
# Load the CSV file into a DataFrame
train_df = pd.read_csv('data/features/data_features_train_sampled_filt.csv')
val_df = pd.read_csv('data/features/data_features_val_sampled_filt.csv')
test_df = pd.read_csv('data/features/data_features_test_sampled_filt.csv')

dfs = {'train': train_df, 'val': val_df, 'test': test_df}
dfs_encoded = {}

for name, df in dfs.items():
  # Perform one-hot encoding using get_dummies()
  encoded_columns = pd.get_dummies(df['target'])

  # Reorder the columns of encoded_columns to match the desired order
  encoded_columns = encoded_columns[labels]

  # Concatenate the one-hot encoded columns with the original DataFrame
  df_encoded = pd.concat([df, encoded_columns], axis=1)

  dfs_encoded[name] = df_encoded

train_df = dfs_encoded['train']
val_df = dfs_encoded['val']
test_df = dfs_encoded['test']


In [6]:
print(train_df.shape,val_df.shape,test_df.shape)

(1393, 12) (299, 12) (299, 12)


In [7]:
train_df.head()

,blinks,right_EAR,left_EAR,blink_duration,blink_frequency,MAR,image,target,alert,slight_drowsy,mod_drowsy,very_drowsy
0,0,0.447605,0.456664,0.100000,12.0,0.165403,alert_3_1791.png,alert,1,0,0,0
1,0,0.564576,0.606079,0.066667,12.0,0.065903,alert_3_1086.png,alert,1,0,0,0
2,0,0.449714,0.461287,0.066667,6.0,0.257237,alert_3_1340.png,alert,1,0,0,0
3,0,0.424942,0.453573,0.083333,12.0,0.074384,alert_3_1522.png,alert,1,0,0,0
4,0,0.524016,0.517682,0.083333,12.0,0.093675,alert_1_1538.png,alert,1,0,0,0


In [8]:
# Create a MinMaxScaler object
scaler = MinMaxScaler()

# Select the columns you want to scale
columns_to_scale = ["right_EAR", "left_EAR", "blink_duration", "blink_frequency", "MAR"]
# Scale the selected columns
train_df[columns_to_scale] = scaler.fit_transform(train_df[columns_to_scale])
test_df[columns_to_scale] = scaler.transform(test_df[columns_to_scale])
val_df[columns_to_scale] = scaler.transform(val_df[columns_to_scale])

In [9]:
train_df.describe()

,blinks,right_EAR,left_EAR,blink_duration,blink_frequency,MAR,alert,slight_drowsy,mod_drowsy,very_drowsy
count,1393.000000,1393.000000,1393.000000,1393.000000,1393.000000,1393.000000,1393.000000,1393.000000,1393.000000,1393.000000
mean,0.061019,0.696229,0.731502,0.093953,0.245234,0.112587,0.251256,0.250538,0.246949,0.251256
std,0.239452,0.126193,0.135148,0.102173,0.207023,0.121407,0.433891,0.433479,0.431392,0.433891
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.645775,0.675974,0.036111,0.059096,0.042816,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.717790,0.753736,0.076399,0.227273,0.072888,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.775151,0.820603,0.132679,0.386943,0.127663,1.000000,1.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [10]:
train_df['target'].value_counts()

alert            350
very_drowsy      350
slight_drowsy    349
mod_drowsy       344
Name: target, dtype: int64

In [11]:
# Save the train, test, and validation DataFrames to separate CSV files
train_df.to_csv('data/features/processed/train_features.csv', index=False)
test_df.to_csv('data/features/processed/test_features.csv', index=False)
val_df.to_csv('data/features/processed/val_features.csv', index=False)

In [3]:
# Save the train, test, and validation DataFrames to separate CSV files
train_df = pd.read_csv('data/features/processed/train_features.csv')
test_df = pd.read_csv('data/features/processed/test_features.csv')
val_df = pd.read_csv('data/features/processed/val_features.csv')

In [5]:
# Create ImageDataGenerators for train, test, and validation sets
train_datagen = ImageDataGenerator(rescale=1.0 / 255)  # Normalize pixel values
test_datagen = ImageDataGenerator(rescale=1.0 / 255)  # Normalize pixel values
val_datagen = ImageDataGenerator(rescale=1.0 / 255)  # Normalize pixel values

# Create train, test, and validation image generators using flow_from_dataframe
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory='data/images/sample/train',
    x_col='image',
    y_col=labels,
    batch_size=32,
    target_size=(224, 224),
    class_mode='raw',
    shuffle=True,
    seed=1
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory='data/images/sample/test',
    x_col='image',
    y_col=labels,
    batch_size=32,
    target_size=(224, 224),
    class_mode='raw',
    shuffle=False,
    seed=1
)

val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    directory='data/images/sample/val',
    x_col='image',
    y_col=labels,
    batch_size=32,
    target_size=(224, 224),
    class_mode='raw',
    shuffle=False,
    seed=1
)


Found 1393 validated image filenames.
Found 299 validated image filenames.
Found 299 validated image filenames.


In [29]:
X,y = train_generator.next()
print(X.shape, y.shape)

(32, 224, 224, 3) (32, 4)


In [18]:
train_df.drop(columns=['image','target']+labels).columns.tolist()

['blinks', 'right_EAR', 'left_EAR', 'blink_duration', 'blink_frequency', 'MAR']

In [6]:
def custom_generator(image_generator, feature_data, batch_size):
    feature_index = 0
    total_samples = len(feature_data)
    while True:
        batch_images, batch_labels = image_generator.next()
        batch_features = feature_data[feature_index : feature_index + batch_size].values
        if (feature_index + batch_size) > total_samples:
            feature_index = 0
        else:
            feature_index = feature_index + batch_size

        yield ([batch_images, batch_features], batch_labels)

# Create custom generators combining image and feature data
train_generator_with_features = custom_generator(train_generator, train_df.drop(columns=['image','target']+labels), batch_size=32)
test_generator_with_features = custom_generator(test_generator, test_df.drop(columns=['image','target']+labels), batch_size=32)
val_generator_with_features = custom_generator(val_generator, val_df.drop(columns=['image','target']+labels), batch_size=32)


In [30]:
[x1,x2],y=next(test_generator_with_features)
print(x1.shape, x2.shape, y.shape)

(32, 224, 224, 3) (32, 6) (32, 4)


In [179]:
# # Load the pre-trained VGG16 model without the top layers
# vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# vgg16.trainable = False # Set all layers in the VGG16 model as non-trainable

# # Define the image input branch
# image_input = Input(shape=(224, 224, 3)) #(720, 1280, 3)
# vgg_output = vgg16(image_input)
# conv1 = Conv2D(32, (3, 3), activation='relu')(vgg_output)
# pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
# # Add more convolutional and pooling layers as needed

# # Define the numerical feature input branch
# feature_input = Input(shape=(6,))
# dense1 = Dense(64, activation='relu')(feature_input)
# # Add more dense layers as needed

# # Concatenate the outputs from both branches
# concatenated = keras.layers.concatenate([Flatten()(pool1), dense1])

# # Add a final dense layer for prediction
# output = Dense(4, activation='softmax')(concatenated)

# # Create the model
# model = Model(inputs=[image_input, feature_input], outputs=output)

# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [167]:
# Define the image input branch
image_input = Input(shape=(224, 224, 3)) #(720, 1280, 3)
conv1 = Conv2D(32, (3, 3), activation='relu')(image_input)
bn1 = BatchNormalization()(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(bn1)
dropout1 = Dropout(0.25)(pool1)
conv2 = Conv2D(16, (3, 3), activation='relu')(dropout1)
bn2 = BatchNormalization()(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(bn2)
dropout2 = Dropout(0.25)(pool2)
conv3 = Conv2D(8, (3, 3), activation='relu')(dropout2)
bn3 = BatchNormalization()(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2))(bn3)

# Define the numerical feature input branch
feature_input = Input(shape=(6,))
dense1 = Dense(64, activation='relu')(feature_input)
dropout3 = Dropout(0.25)(dense1)
dense2 = Dense(32, activation='relu')(dropout3)
dropout4 = Dropout(0.25)(dense2)
dense3 = Dense(16, activation='relu')(dropout4)

# Concatenate the outputs from both branches
concatenated = keras.layers.concatenate([Flatten()(pool3), dense3])

# Add a final dense layer for prediction
output = Dense(4, activation='softmax')(concatenated)

# Create the model
model = Model(inputs=[image_input, feature_input], outputs=output)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Learning rate scheduler
def lr_scheduler(epoch):
    initial_lr = 0.001
    drop_rate = 0.5
    epochs_drop = 5
    lr = initial_lr * (drop_rate ** (epoch // epochs_drop))
    return lr

scheduler = LearningRateScheduler(lr_scheduler)

In [7]:
# Define the image input branch
image_input = Input(shape=(224, 224, 3)) #(720, 1280, 3)
conv1 = Conv2D(32, (3, 3), activation='relu')(image_input)
bn1 = BatchNormalization()(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(bn1)
conv2 = Conv2D(16, (3, 3), activation='relu')(pool1)
bn2 = BatchNormalization()(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(bn2)
conv3 = Conv2D(8, (3, 3), activation='relu')(pool2)
bn3 = BatchNormalization()(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2))(bn3)

# Define the numerical feature input branch
feature_input = Input(shape=(6,))
dense1 = Dense(64, activation='relu')(feature_input)
dense2 = Dense(32, activation='relu')(dense1)
dense3 = Dense(16, activation='relu')(dense2)

# Concatenate the outputs from both branches
concatenated = keras.layers.concatenate([Flatten()(pool3), dense3])

# Add a final dense layer for prediction
output = Dense(4, activation='softmax')(concatenated)

# Create the model
model = Model(inputs=[image_input, feature_input], outputs=output)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Learning rate scheduler
def lr_scheduler(epoch):
    initial_lr = 0.001
    drop_rate = 0.5
    epochs_drop = 5
    lr = initial_lr * (drop_rate ** (epoch // epochs_drop))
    return lr

scheduler = LearningRateScheduler(lr_scheduler)

In [8]:
# fit network
history = model.fit(train_generator_with_features,
                    epochs=4,
                    steps_per_epoch=len(train_generator),
                    validation_data=val_generator_with_features,
                    validation_steps= len(val_generator),
                    callbacks=[scheduler])

# 29 mins con 3 epochs => model_3
# 6 mins 34s solo he probado primero con 1 epoch => model_4
# de aquí para abajo con vscode
# 34.1s, 1 epoch, en vscode en vez de google que tarda por tener que ir a drive => model_5
# 8 mins y pico, 1 epoch, con vgg16 => pero ha fallado en la validación al concatenar, estaba tardando mucho tampoco lo iba a usar

# en todo lo de arriba estaban mal los generators, por eso ha fallado el vgg16

# 13 mins 28.8s, ordenador sobrecalentado (por eso tarda), 5 epochs, modelo con más capas => model_6

# 22 mins 46.5s, sigue caliente, 10 epochs y poniendo el epochs drop a 5 que antes no hacía nada, modelo del 6, shuffle true en train_gen => model_7
# al hacer el shuffle ya consigue aprender de los datos de train, pero hay que poner dropout porque en validation va mal
# no parece que el lr esté cambiando, vale es que no lo había ejecutado con el cambio
# mierda he borrado el 7 sin querer, (el que pone 7 es el 8 del model)

# 28 mins 11.6s, sigue caliente, 10 epochs, con el epoch drop a 5 ahora sí, añadiendo dropout => model_8
# el dropout no parece que haga mucho, estoy llegando a accuracy 1 en train y val sigue bajo

# 10 mins solo 1 epoch, sigue caliente, vgg16, 4 epochs por probar, con los arreglos de los generators y el shuffle que antes igual no aprendía porque estaba mal => model_9
# ETA del primero ya son como 9 mins, no lo voy a dejar hasta el final, solo quiero ver el val_accuracy como es

# 4 mins 50.8s, 4 epochs, con las nuevas fotos, sin dropout => model_10

Epoch 1/4
44/44 [==============================] - 74s 2s/step - loss: 0.4110 - accuracy: 0.8457 - val_loss: 1.4920 - val_accuracy: 0.2508 - lr: 0.0010
Epoch 2/4
44/44 [==============================] - 70s 2s/step - loss: 0.0594 - accuracy: 0.9785 - val_loss: 1.3516 - val_accuracy: 0.4348 - lr: 0.0010
Epoch 3/4
44/44 [==============================] - 80s 2s/step - loss: 0.0520 - accuracy: 0.9792 - val_loss: 1.4118 - val_accuracy: 0.3980 - lr: 0.0010
Epoch 4/4
44/44 [==============================] - 66s 2s/step - loss: 0.0098 - accuracy: 0.9964 - val_loss: 1.6656 - val_accuracy: 0.3913 - lr: 0.0010


In [9]:
model.save('models/model_10.h5')

In [10]:
np.save('history_10.npy', history.history)

In [11]:
# model = load_model('models/model_1.h5') // model 1 y 2 primeros (1 sin val y 2 con val), con los 9000 y val/test acc 1, epoch 1
# model 3 es con el sample y 3 epochs

In [12]:
prediction = model.predict(test_generator_with_features, steps=len(test_generator))
# 4 min 56s => model_3
# 2s => model_4
# 2.6s => model_5
# 13.9s => model_6 (ordenador sobrecalentado no sé si afectará)
# 8.4s => model_7  (sigue caliente)
# 3.3s => model_8 (sigue caliente)
# 5.7s => model_10

10/10 [==============================] - 5s 545ms/step


In [17]:
prediction.shape

(299, 4)

In [13]:
test_pred = np.argmax(prediction, axis=1)

In [19]:
test_pred.shape

(299,)

In [14]:
test_true_values = test_df[labels].to_numpy()

In [15]:
test_true = np.argmax(test_true_values, axis=1)
test_true.shape

(299,)

In [16]:
accuracy = accuracy_score(test_true, test_pred)
accuracy
# model_3
# en test sigue teniendo un 100% accuracy ?????
# añadir dropout o regularización?
# igual sigue siendo demasiado parecido, durante el mes ir cargando el resto de mins de los vídeos
# intentar liar a alguien para grabarles también

# model_4
# ya da un accuracy malo como quería
# añadir más capas a la red (maybe vgg16)
# y también más epochs que solo he puesto 1
# 0.18 o así

# model_6
# 0.284

# model_7
# 0.3545

# model_8
# 0.197

0.3779264214046823

In [17]:
print(classification_report(test_true,test_pred, target_names=labels, digits=4))


               precision    recall  f1-score   support

        alert     0.0000    0.0000    0.0000        75
slight_drowsy     0.2925    0.9867    0.4512        75
   mod_drowsy     0.0000    0.0000    0.0000        74
  very_drowsy     0.8478    0.5200    0.6446        75

     accuracy                         0.3779       299
    macro avg     0.2851    0.3767    0.2740       299
 weighted avg     0.2860    0.3779    0.2749       299



c:\Users\carlo\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\carlo\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\carlo\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

In [18]:
# Construct a visualization for the confusion matrix
z_test = confusion_matrix(test_true,test_pred)

# reformat the matrix
z_test[[0,1,2,3],:] = z_test[[3,2,1,0],:]

x = ['alert','slight_drowsy','mod_drowsy','very_drowsy']
y = ['very_drowsy','mod_drowsy', 'slight_drowsy', 'alert']

def format_text(z):
    """
    This function receives the confusion matrix values and formats the text
    to display in the matrix with the value and percentage of total data.
    """
    return f'{z}<br>({z / np.sum(z_test) * 100:.1f}%)'

z_text = np.vectorize(format_text)(z_test)

heatmap = ff.create_annotated_heatmap(z_test, x=x, y=y, annotation_text=z_text, colorscale='plasma')
heatmap.update_layout(height=350,width=500,
                      xaxis_title="Predicted Label",yaxis_title="True Label")
heatmap.show()

In [127]:
print(test_true)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3]


In [128]:
print(test_pred)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 2 2 2 2 2 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 1 2 2 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2
 2 1 0 0 0 1 1 3 0 1 0 0 0 1 1 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 1
 0 1 1 1 0 1 0 1 0 1 1 1 2 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 1 0 0 0 1 0 0
 2 3 2 2 2 2 3 2 2 3 2 2 2 2 3 2 2 2 2 3 2 3 2 2 1 2 2 2 2 2 2 2 2 2 2 3 2
 2 2 2 2 2 3 2 1 2 2 3 2 2 3 2 2 2 2 3 2 1 2 2 2 2 2 2 2 2 3 1 2 2 2 3 2 2
 2 2 2]
